In [1]:
!pip install grape

  Using cached grape-0.2.2.tar.gz (9.2 kB)
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Using cached bioregistry-0.10.66-py3-none-any.whl (10.4 MB)
  Using cached py_cpuinfo-9.0.0-py3-none-any.whl (22 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.2/29.2 MB 18.3 MB/s eta 0:00:0000:0100:01
  Using cached embiggen-0.11.83.tar.gz (240 kB)
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.6/20.6 MB 20.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 21.4 MB/s eta 0:00:0000:0100:01
  Using cached tqdm-4.66.1-py3-none-any.whl (78 kB)
  Using cached humanize-4.8.0-py3-none-any.whl (117 kB)
  Using cached matplotlib-3.8.0-cp311-cp311-macosx_10_12_x86_64.whl (7.6 MB)
  Using cached scikit_learn-1.3.2-cp311-cp311-macosx_10_9_x86_64.whl (10.1 MB)
  Using cached dict_hash-1.1.32.tar.gz (7.0 kB)
  Preparing metadata (setup.py) ... done
  Using cached userinput-1.0.21.tar.gz (8

In [1]:
from grape.datasets.kghub import KGIDG
import pandas as pd
from grape.edge_prediction import edge_prediction_evaluation
from grape.edge_prediction import PerceptronEdgePrediction
from grape.embedders import FirstOrderLINEEnsmallen, SecondOrderLINEEnsmallen
from grape.embedders import DeepWalkCBOWEnsmallen, DeepWalkSkipGramEnsmallen
from grape.embedders import WalkletsCBOWEnsmallen, WalkletsSkipGramEnsmallen

/Users/jtr4v/PythonProject/dans_kg_idg/venv/lib/python3.11/site-packages/ensmallen/__init__.py:22: UserWarning: Ensmallen is compiled for the Intel Haswell architecture (2013).On the current machine, the flags '['avx2', 'bmi2', 'popcnt']' are required but '{'avx2', 'bmi2'}' are not available.
The library will use a slower but more compatible version (Intel Core2 2006).
  warnings.warn(


In [2]:
g = KGIDG(version='20230601')
main_component = g.remove_components(top_k_components=1)
dense_main_component = main_component.remove_dendritic_trees()
drug_types = ["biolink:ChemicalSubstance", "biolink:ChemicalEntity", "biolink:Drug"]
protein_types = ["biolink:Protein"]
minority_edge_type = 'minority_edge'
dense_main_component.replace_edge_type_name_from_edge_node_type_names_inplace(
    edge_type_name=minority_edge_type,
    source_node_type_names=drug_types,
    destination_node_type_names=protein_types
)
dense_main_component

<div class="graph-report"><style>.graph-report li {margin: 0.5em 0 0.5em 0;}.graph-report .paragraph {text-align: justify;word-break: break-all;}.graph-report .small-columns {column-count: 4;column-gap: 2em;}.graph-report .medium-columns {column-count: 3;column-gap: 2em;}.graph-report .large-columns {column-count: 2;column-gap: 2em;}.graph-report .single-column {}@media only screen and (max-width: 600px) {.graph-report .small-columns {column-count: 1;}.graph-report .medium-columns {column-count: 1;}.graph-report .large-columns {column-count: 1;}}@media only screen and (min-width: 600px) and (max-width: 800px) {.graph-report .small-columns {column-count: 2;}.graph-report .medium-columns {column-count: 1;}.graph-report .large-columns {column-count: 1;}}@media only screen and (min-width: 800px) and (max-width: 1024px) {.graph-report .small-columns {column-count: 3;}.graph-report .medium-columns {column-count: 2;}.graph-report .large-columns {column-count: 1;}}</style><h2>KGIDG</h2><p class="paragraph">The undirected multigraph KGIDG has 239.46K heterogeneous nodes and 1.01M heterogeneous edges. The graph is connected, that is, it is composed of a single connected component that includes all nodes and edges. The RAM requirements for the nodes and edges data structures are 28.57MB and 9.97MB respectively.</p><h3 style="margin: 1em 0 0 0;">Degree centrality</h3><p class="paragraph">The minimum node degree is 2, the maximum node degree is 9.36K, the mode degree is 2, the mean degree is 8.41 and the node degree median is 4.</p><p class="paragraph">The nodes with the highest degree centrality are <a href='http://amigo.geneontology.org/amigo/term/GO:0065007' target='_blank' title='Go to Gene Ontology to get more informations about GO:0065007'>GO:0065007</a> (degree 9.36K and node types <a href='https://biolink.github.io/biolink-model/docs/BiologicalProcess.html' target='_blank' title='Go to BioLink to get more informations about biolink:BiologicalProcess'>biolink:BiologicalProcess</a> and <a href='https://biolink.github.io/biolink-model/docs/NamedThing.html' target='_blank' title='Go to BioLink to get more informations about biolink:NamedThing'>biolink:NamedThing</a>), <a href='https://www.ebi.ac.uk/chebi/searchId.do?chebiId=CHEBI:24995' target='_blank' title='Go to CHEBI to get more informations about CHEBI:24995'>CHEBI:24995</a> (degree 7.34K and node type <a href='https://biolink.github.io/biolink-model/docs/ChemicalSubstance.html' target='_blank' title='Go to BioLink to get more informations about biolink:ChemicalSubstance'>biolink:ChemicalSubstance</a>), <a href='https://www.ebi.ac.uk/chebi/searchId.do?chebiId=CHEBI:52898' target='_blank' title='Go to CHEBI to get more informations about CHEBI:52898'>CHEBI:52898</a> (degree 7.26K and node type <a href='https://biolink.github.io/biolink-model/docs/ChemicalSubstance.html' target='_blank' title='Go to BioLink to get more informations about biolink:ChemicalSubstance'>biolink:ChemicalSubstance</a>), <a href='https://www.ebi.ac.uk/chebi/searchId.do?chebiId=CHEBI:30616' target='_blank' title='Go to CHEBI to get more informations about CHEBI:30616'>CHEBI:30616</a> (degree 6.00K and node types <a href='https://biolink.github.io/biolink-model/docs/ChemicalSubstance.html' target='_blank' title='Go to BioLink to get more informations about biolink:ChemicalSubstance'>biolink:ChemicalSubstance</a> and <a href='https://biolink.github.io/biolink-model/docs/ChemicalEntity.html' target='_blank' title='Go to BioLink to get more informations about biolink:ChemicalEntity'>biolink:ChemicalEntity</a>) and <a href='https://www.ebi.ac.uk/chebi/searchId.do?chebiId=CHEBI:456216' target='_blank' title='Go to CHEBI to get more informations about CHEBI:456216'>CHEBI:456216</a> (degree 5.43K and node types <a href='https://biolink.github.io/biolink-model/docs/ChemicalSubstance.html' target='_blank' title='Go to BioLink to get more informations about biolink:ChemicalSubstance'>biolink:ChemicalSubstance</a> and <a

In [ ]:
# Set smoke test to True for testing:
# SMOKE_TEST = True
NUMBER_OF_HOLDOUTS = 10
VALIDATION_UNBALANCE_RATES = (1.0, )
TRAIN_SIZES = (0.75,)
subgraph = dense_main_component.filter_from_names(
    edge_type_names_to_keep=[minority_edge_type]
)
# Disable the smoke test when you need to run the real thing:
SMOKE_TEST = False # <- I WILL CHECK THIS ONCE AGAIN, BUT I AM PRETTY SURE I RAN IT WITH FALSE
NUMBER_OF_HOLDOUTS = 10
VALIDATION_UNBALANCE_RATES = (1.0, )
TRAIN_SIZES = (0.75,)
results = []
for train_size in TRAIN_SIZES:
    for validation_use_scale_free in (True, False):
        for ModelClass in  [
                FirstOrderLINEEnsmallen, SecondOrderLINEEnsmallen,
                DeepWalkCBOWEnsmallen, DeepWalkSkipGramEnsmallen,
                WalkletsCBOWEnsmallen, WalkletsSkipGramEnsmallen,
            ]:
            # If the embedding method involves edge sampling, we train a run
            # using the scale free and one using the uniform.
            if "use_scale_free_distribution" in ModelClass().parameters():
                parameter_sets = [
                    dict(
                        use_scale_free_distribution = True
                    ),
                    dict(
                        use_scale_free_distribution = False
                    )
                ]
            else:
                parameter_sets = [dict()]
            print(f"train size: {train_size}; validation_use_scale_free: {validation_use_scale_free} ModelClass {ModelClass}")
            for parameter_set in parameter_sets:
                results.append(edge_prediction_evaluation(
                    smoke_test=SMOKE_TEST,
                    holdouts_kwargs=dict(
                        train_size=train_size,
                        edge_types=[minority_edge_type],
                    ),
                    evaluation_schema="Connected Monte Carlo",
                    node_features=ModelClass(**parameter_set),
                    graphs=dense_main_component,
                    models=[
                        PerceptronEdgePrediction(
                            edge_features=None,
                            edge_embeddings="Hadamard",
                            number_of_edges_per_mini_batch=32,
                            use_scale_free_distribution=use_scale_free_distribution
                        )
                        for use_scale_free_distribution in (True, False)
                    ],
                    enable_cache=True,
                    number_of_holdouts=NUMBER_OF_HOLDOUTS,
                    use_scale_free_distribution=validation_use_scale_free,
                    validation_unbalance_rates=VALIDATION_UNBALANCE_RATES,
                    subgraph_of_interest=subgraph,
                    use_subgraph_as_support=False
                ))
results = pd.concat(results)
results.to_csv("kg_idg_negative_select.tsv",sep="\t")

train size: 0.75; validation_use_scale_free: True ModelClass <class 'embiggen.embedders.ensmallen_embedders.first_order_line.FirstOrderLINEEnsmallen'>


Evaluating on KGIDG:   0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]